<a href="https://colab.research.google.com/github/MaximeGloesener/HandsOnAI-Challenge1/blob/master/RealTimeTreatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Hardware Informations (GPU)**

In [ ]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

# **2. Importation of librairies**

In [ ]:
!pip install tf-explain

In [ ]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np 
import os
import cv2
import csv
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions #224*224
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance
from google.colab import files
from tqdm import tqdm
from google.colab.patches import cv2_imshow
print("Tensorflow version: "+tf.__version__)
print("Keras version: " + tf.keras.__version__)

#**3. Connexion à Google Colab pour acceder aux bases de données**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

**4. Create the labels file "classes.txt"**

In [ ]:
!printf '%s\n' 'fire' 'no_fire' 'start_fire'> classes.txt

#**5. Training parameters and selectioon of Pretrained model**

In [ ]:
# Fix random seed 
tf.keras.utils.set_random_seed(42)

In [ ]:
nb_classes = 3
nbr_batch_size=64 #@param [1,2,4,8,16,32,64,128] {type:"raw"}
data_path= "gdrive/MyDrive/Challenge1"
input_dim=224 #@param [224,299] {type:"raw"}  
dataset_name='db1' #@param ["db1","db2"]
dataset_path = os.path.join(data_path,dataset_name)
classes_path = "classes.txt"
csv_path = 'result.csv'

In [ ]:
print(dataset_path)

# **6. Get the number of classes**

In [ ]:
# Get the class names
with open(classes_path, 'r') as f:
    classes = f.readlines()
    classes = list(map(lambda x: x.strip(), classes))
num_classes = len(classes)

In [ ]:
print(num_classes)

#**14. Load model**


In [ ]:
model = load_model('gdrive/MyDrive/Challenge1/test.h5')

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(os.path.join(data_path,'test_data/test/Test_Dataset')
	,          # chemin vers le jeu de données
	seed=42,                    # Initialisation du générateur aléatoire (permutations)
	image_size=(input_dim,input_dim),       # Taille des images d'entrée
	batch_size=nbr_batch_size,      # Taille du mini-batch
  label_mode='categorical'    # Conversion au format One-Hot
)

In [ ]:
score = model.evaluate(test_ds,  steps=len(test_ds),workers = 1)
print("%s: %.2f%%" % (model.metrics_names[0], score[0]))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#**16. Test the model with a test image**

In [ ]:
%matplotlib inline
classes = test_ds.class_names
image_path =  os.path.join(data_path,'test_data/Test_Dataset1/Test_Dataset/fire/test001.jpg')
img = Image.open(image_path).convert('RGB')
x = tf.keras.utils.img_to_array(img,data_format='channels_last')
x = tf.keras.preprocessing.image.smart_resize(x, size=(input_dim,input_dim))
x = np.expand_dims(x, axis=0)
# predict
pred = model.predict(x,batch_size=1)[0]

for (pos,prob) in enumerate(pred):
    class_name = classes[pos]
    if (pos == np.argmax(pred)) :
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        font = cv2.FONT_HERSHEY_COMPLEX 
        textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
        textX = (img.shape[1] - textsize[0]) / 2
        textY = (img.shape[0] + textsize[1]) / 2
        cv2.putText(img, class_name, (int(textX)-10, int(textY)), font, 2, (255,0,0), 6, cv2.LINE_AA)
        plt.imshow(img)
    print("Class Name : %s" % (class_name), "---", "Class Probability: %.2f%%" % (prob*100))
plt.show()

In [ ]:
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/fire_detection/TestVideo1.avi

In [ ]:
###!find . -type f -iname \*.jpg -delete

In [ ]:
video_cap = cv2.VideoCapture('TestVideo1.avi')

frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video_cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'DivX')
output = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

while True:
        success, frame = video_cap.read()

        if success:

          img = cv2.resize(frame, (input_dim, input_dim), interpolation = cv2.INTER_AREA)

          # Resizing into dimensions you used while training
          img = tf.keras.preprocessing.image.img_to_array(img)
          img = np.expand_dims(img, axis = 0)

          pred = model.predict(img, batch_size = 1)[0]

          # Annotate the frame
          for (pos,prob) in enumerate(pred):
              class_name = classes[pos]
              if (pos == np.argmax(pred)) :
                img = cv2.imread(image_path)
                img = cv2.resize(img, (input_dim, input_dim), interpolation = cv2.INTER_AREA)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                font = cv2.FONT_HERSHEY_COMPLEX 

                textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
                textX = (img.shape[1] - textsize[0]) / 2
                textY = (img.shape[0] + textsize[1]) / 2
                cv2.putText(img, class_name, (int(textX)-10, int(textY)), font, 1, (255,255,255), 2, cv2.LINE_AA)
                output.write(img)

        else:
          break

video_cap.release()
output.release()
cv2.destroyAllWindows()